Idea for harmonizing a melody. 

1. Take some melody, like the following:

In [14]:
from fractions import Fraction
from harmonica.pitch._melody import PitchSeq
from harmonica.time._event._clip import Clip, NoteClip
from harmonica.time._event._event import Note

# Pitch sequence
pitch_seq = PitchSeq([0, -2, 1, 2, 0, 3, 4, 5, 3, 2, 0, 3]) + 60

# Build into melody by giving rhythm
melody = NoteClip(
    [
        Note(pitch, onset, duration)
        for pitch, onset, duration in zip(
            pitch_seq.pitches,
            [
                Fraction("1/2"),
                1 + Fraction("1/4"),
                "2",
                4 + Fraction("1/2"),
                5 + Fraction("1/4"),
                "6",
                "7",
                8 + Fraction("1/2"),
                9 + Fraction("1/4"),
                "10",
                10 + Fraction("3/4"),
                11 + Fraction("1/2"),
            ],
            [
                Fraction("3/4"),
                Fraction("3/4"),
                "2",
                Fraction("3/4"),
                Fraction("3/4"),
                "1",
                "1",
                Fraction("3/4"),
                Fraction("3/4"),
                Fraction("3/4"),
                Fraction("3/4"),
                4 + Fraction("1/2"),
            ],
        )
    ]
)

# Clip([melody]).preview()

2. Take some harmonic rhythm, like the following:

In [15]:
harmonic_rhythm = [Fraction(onset) for onset in [0, 2, 4, 6, 7, 8, 10, 12]]

3. Treat each onset in the harmonic rhythm as a window of time from which to group together notes and fit a scale to them. 

For example, the first window would be for all notes with onsets `0 <= onset < 2`. In our melody, that's the pitch class set `{0, 10}`.

Then the next window would be for all notes with onset `2 <= onset < 4`. That's the pitch class set `{1}`.

So it should take a note clip representing a melody and a clip representing a harmonic rhythm and return a clip of pitch class sets (`ScaleChangeClip`).

In [ ]:
from harmonica.pitch._pitchset import PitchSet
from harmonica.time import ScaleChangeClip, NoteClip, Clip
from harmonica.time._event._event import ScaleChange


def sample_pcsets_from_note_clip(
    note_clip: NoteClip, sample_rhythm: list[Fraction], modulus: int
) -> ScaleChangeClip:
    assert modulus > 0, "Modulus must be positive."

    result = ScaleChangeClip()

    appended = 0

    for i in range(len(sample_rhythm) - 1):
        pitches = []
        for note in note_clip.get_notes()[appended:]:
            if note.onset >= sample_rhythm[i + 1]:
                if pitches:
                    result.add_event(
                        ScaleChange(
                            sample_rhythm[i],
                            PitchSet(sorted(set(pitches))).classify(modulus),
                        )
                    )
                break
            pitches.append(note.pitch)
            appended += 1

    return result


changes = sample_pcsets_from_note_clip(melody, harmonic_rhythm, 12)
for change in changes.events:
    print(change)

ScaleChange(onset=0, scale=PitchClassSet(pitch_classes=[0, 10], modulus=12, root=None))
ScaleChange(onset=2, scale=PitchClassSet(pitch_classes=[1], modulus=12, root=None))
ScaleChange(onset=4, scale=PitchClassSet(pitch_classes=[0, 2], modulus=12, root=None))
ScaleChange(onset=6, scale=PitchClassSet(pitch_classes=[3], modulus=12, root=None))
ScaleChange(onset=7, scale=PitchClassSet(pitch_classes=[4], modulus=12, root=None))
ScaleChange(onset=8, scale=PitchClassSet(pitch_classes=[3, 5], modulus=12, root=None))
